# 3일차 1교시 스파크 기본 명령어 이해

### 목차
* 1. 스파크를 통한 CSV 파일 읽기
* 2. 스파크의 2가지 프로그래밍 방식 비교
* 3. 스파크를 통한 JSON 파일 읽기
* 4. 뷰 테이블 생성 및 조회
* 5. 참고 자료
  * [Spark Programming Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
  * <a href="https://spark.apache.org/docs/2.4.5/api/sql/" target="_blank">PySpark 2.4.5 Builtin Functions</a>

## 1. 스파크를 통한 CSV 파일 읽기
> Spark 2.4.5 버전을 기준으로 작성되었습니다. 스파크는 2.0 버전으로 업데이트 되면서 DataFrames 은 Datasets 으로 통합되었고, 기존의 RDD 에서 사용하던 연산 및 기능과 DataFrame 에서 사용하던 것 모두 사용할 수 있습니다. 스파크 데이터 모델은 RDD (Spark1.0) —> Dataframe(Spark1.3) —> Dataset(Spark1.6) 형태로 업그레이드 되었으나, 본문에서 일부 DataFrames 와 DataSets 가 거의 유사하여, 일부 혼용되어 사용되는 경우가 있을 수 있습니다.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession \
    .builder \
    .config("spark.sql.session.timeZone", "Asia/Seoul") \
    .getOrCreate()

In [9]:
# !which python
!/opt/conda/bin/python --version

print("spark.version: {}".format((spark.version)))
spark

Python 3.8.6
spark.version: 3.0.1


In [16]:
strings = spark.read.text("../requirements.txt")
strings.show(5, truncate=False)
count = strings.count()
print("count of word is {}".format(count))

+--------+
|value   |
+--------+
|boto3   |
|scrapy  |
|selenium|
|mrjob   |
|pyspark |
+--------+
only showing top 5 rows

count of string is 9


### 아무런 옵션을 주지 않는 경우 스파크가 알아서 컬럼 이름과 데이터 타입을 (string) 지정합니다

In [3]:
log_access = spark.read.csv("data/log_access.csv")
log_access.printSchema()
log_access.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)

+----------+-----+------+
|       _c0|  _c1|   _c2|
+----------+-----+------+
|    a_time|a_uid|  a_id|
|1603645200|    1| login|
|1603647200|    1|logout|
|1603649200|    2| login|
|1603650200|    2|logout|
|1603653200|    2| login|
|1603657200|    3| login|
|1603659200|    3|logout|
|1603660200|    4| login|
|1603664200|    4|logout|
|1603664500|    4| login|
|1603666500|    5| login|
|1603669500|    5|logout|
|1603670500|    6| login|
|1603673500|    7| login|
|1603674500|    8| login|
|1603675500|    9| login|
+----------+-----+------+



### 첫 번째 라인에 헤더가 포함되어 있는 경우 아래와 같이 header option 을 지정하면 컬럼 명을 가져올 수 있습니다

In [5]:
log_access = spark.read.option("header", "true").csv("data/log_access.csv")
log_access.printSchema()
log_access.show()

root
 |-- a_time: string (nullable = true)
 |-- a_uid: string (nullable = true)
 |-- a_id: string (nullable = true)

+----------+-----+------+
|    a_time|a_uid|  a_id|
+----------+-----+------+
|1603645200|    1| login|
|1603647200|    1|logout|
|1603649200|    2| login|
|1603650200|    2|logout|
|1603653200|    2| login|
|1603657200|    3| login|
|1603659200|    3|logout|
|1603660200|    4| login|
|1603664200|    4|logout|
|1603664500|    4| login|
|1603666500|    5| login|
|1603669500|    5|logout|
|1603670500|    6| login|
|1603673500|    7| login|
|1603674500|    8| login|
|1603675500|    9| login|
+----------+-----+------+



### inferSchema 옵션으로 데이터 값을 확인하고 스파크가 데이터 타입을 추정하게 할 수 있습니다

In [6]:
log_access = spark.read.option("header", "true").option("inferSchema", "true").csv("data/log_access.csv")
log_access.printSchema()
log_access.show()

root
 |-- a_time: integer (nullable = true)
 |-- a_uid: integer (nullable = true)
 |-- a_id: string (nullable = true)

+----------+-----+------+
|    a_time|a_uid|  a_id|
+----------+-----+------+
|1603645200|    1| login|
|1603647200|    1|logout|
|1603649200|    2| login|
|1603650200|    2|logout|
|1603653200|    2| login|
|1603657200|    3| login|
|1603659200|    3|logout|
|1603660200|    4| login|
|1603664200|    4|logout|
|1603664500|    4| login|
|1603666500|    5| login|
|1603669500|    5|logout|
|1603670500|    6| login|
|1603673500|    7| login|
|1603674500|    8| login|
|1603675500|    9| login|
+----------+-----+------+



## 2. 스파크의 2가지 프로그래밍 방식 비교

### 하나. 구조화된 API 호출을 통해 데이터를 출력하는 방법
> 출력 시에 bigint 값인 날짜는 아래와 같이 from_unixtime 및 to_timestamp 함수를 통해 변환할 수 있습니다.

In [8]:
from pyspark.sql.functions import unix_timestamp, from_unixtime, to_timestamp, to_date, col, lit

df = spark.read.option("inferSchema", "true").json("data/activity-data")

# 구조화된 API 를 통한 구문
timestamp = df.select(
    "Arrival_Time",
    to_timestamp(from_unixtime(col('Arrival_Time') / lit(1000)), 'yyyy-MM-dd HH:mm:ss').alias('String_Datetime')
)
timestamp.show(5)

+-------------+-------------------+
| Arrival_Time|    String_Datetime|
+-------------+-------------------+
|1424686734992|2015-02-23 19:18:54|
|1424686735190|2015-02-23 19:18:55|
|1424686735395|2015-02-23 19:18:55|
|1424686735593|2015-02-23 19:18:55|
|1424686735795|2015-02-23 19:18:55|
+-------------+-------------------+
only showing top 5 rows



### 둘. 표현식 형식으로 그대로 사용하여 출력하는 방법
> 컬럼(col) 혹은 함수(concat 등)를 직접 사용하는 방식을 **구조화된 API** 를 사용한다고 말하고 SQL 구문으로 표현하는 방식을 **SQL 표현식**을 사용한다고 말합니다

In [11]:
# SQL Expression 통한 구문
ts = df.selectExpr(
    "Arrival_Time",
    "to_timestamp(from_unixtime(Arrival_Time / 1000), 'yyyy-MM-dd HH:mm:ss') as String_Datetime"
)
ts.show(5)

+-------------+-------------------+
| Arrival_Time|    String_Datetime|
+-------------+-------------------+
|1424686734992|2015-02-23 19:18:54|
|1424686735190|2015-02-23 19:18:55|
|1424686735395|2015-02-23 19:18:55|
|1424686735593|2015-02-23 19:18:55|
|1424686735795|2015-02-23 19:18:55|
+-------------+-------------------+
only showing top 5 rows



#### Select 뿐만 아니라 filter 의 경우도 Expression 을 사용할 수 있습니다

In [12]:
df.filter(col("index") > 100).select("index", "user").groupBy("user").count().show()
# 대부분의 구문에서 표현식을 통해 처리할 수 있도록 내부적으로 2가지 방식에 대해 모두 구현되어 있습니다. 
df.filter("index > 100").select("index", "user").groupBy("user").count().show()

+----+-----+
|user|count|
+----+-----+
|   g|91650|
|   f|92030|
|   e|96000|
|   h|77300|
|   d|81220|
|   c|77130|
|   i|92530|
|   b|91210|
|   a|80824|
+----+-----+

+----+-----+
|user|count|
+----+-----+
|   g|91650|
|   f|92030|
|   e|96000|
|   h|77300|
|   d|81220|
|   c|77130|
|   i|92530|
|   b|91210|
|   a|80824|
+----+-----+



## 3. 스파크를 통한 JSON 파일 읽기

In [13]:
json = spark.read.json("data/activity-data")
users = json.filter("index > 100").select("index", "user").groupBy("user").count()
users.show(5)

+----+-----+
|user|count|
+----+-----+
|   g|91650|
|   f|92030|
|   e|96000|
|   h|77300|
|   d|81220|
+----+-----+
only showing top 5 rows



## 4. 뷰 테이블 생성 및 조회
> 이미 생성된 데이터 프레임을 통해서 현재 세션에서만 조회 가능한 임시 뷰 테이블을 만들어 SQL 질의가 가능합니다.

In [14]:
users.createOrReplaceTempView("users")
spark.sql("select * from users where count is not null and count > 9000 order by count desc").show(5)

+----+-----+
|user|count|
+----+-----+
|   e|96000|
|   i|92530|
|   f|92030|
|   g|91650|
|   b|91210|
+----+-----+
only showing top 5 rows



### JSON 파일을 읽는 여러가지 방법

In [15]:
# 스키마 확인 - 3가지 모두 동일한 결과를 얻을 수 있으며 편한 방식을 선택하시면 됩니다
df = spark.read.format("json").load("./data/flight-data/json/2015-summary.json") # 미국 교통통계국이 제공하는 항공운항 데이터
df.printSchema()

df2 = spark.read.load("./data/flight-data/json/2015-summary.json", format="json")
df2.printSchema()

df3 = spark.read.json("./data/flight-data/json/2015-summary.json")
df3.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



## 5. 스파크 애플리케이션의 개념 이해

| 구분 | 설명 | 기타 |
|---|---|---|
| Application | 스파크 프레임워크를 통해 빌드한 프로그램. 전체 작업을 관리하는 Driver 와 Executors 상에서 수행되는 프로그램으로 구분합니다 | - |
| SparkSession | 스파크의 모든 기능을 사용하기 위해 생성하는 객체 | - |
| Job | 하나의 액션(save, collect 등)을 수행하기 위해 여러개의 타스크로 구성된 병렬처리 단위 | DAG 혹은 Spark Execution Plan |
| Stage | 하나의 잡은 다수의 스테이지라는 것으로 구성되며, 하나의 스테이지는 다수의 타스크 들로 구성됩니다 | - |
| Task | 스파크 익스큐터에 보내지는 하나의 작업 단위 | 하나의 Core 혹은 Partition 단위의 작업 |

## 8. 실습문제

### 실습#1. data/tbl_user.csv 파일을 읽고, 올바른 데이터 타입을 추정할 수 있도록 옵션을 주어 데이터를 읽고, 해당 스키마를 출력하세요
> 참고 : printSchema()

In [16]:
user = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_user.csv")
user.printSchema()

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)



### 실습#2. data/tbl_purchase.csv 파일을 읽고, 데이터를 출력하되 p_time 필드는 날짜 함수를 이용하여 식별 가능하도록 데이터를 출력하세요
> 참고 : from_unixtime(column), show()

In [17]:
purchase = spark.read.option("header", "true").option("inferSchema", "true").csv("data/tbl_purchase.csv")
purchase.selectExpr("*", "from_unixtime(p_time)").show(truncate=False)

+----------+-----+----+-----------+--------+----------------------------------------------------------+
|p_time    |p_uid|p_id|p_name     |p_amount|from_unixtime(CAST(p_time AS BIGINT), yyyy-MM-dd HH:mm:ss)|
+----------+-----+----+-----------+--------+----------------------------------------------------------+
|1603651550|1    |2000|LG DIOS    |2000000 |2020-10-26 03:45:50                                       |
|1603694755|1    |2000|LG Gram    |1800000 |2020-10-26 15:45:55                                       |
|1603673500|2    |2001|LG Cyon    |1400000 |2020-10-26 09:51:40                                       |
|1603652155|3    |2002|LG TV      |1000000 |2020-10-26 03:55:55                                       |
|1603674500|4    |2003|LG Computer|4500000 |2020-10-26 10:08:20                                       |
|1603665955|5    |2004|LG Gram    |3500000 |2020-10-26 07:45:55                                       |
|1603666155|5    |2004|LG TV      |2500000 |2020-10-26 07:49:15 

## 9. 질문과 답변

### Q1. 하나의 물리적인 장비 혹은 JVM 위에서 여러개의 SparkSession 실행이 가능한가?
